In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('./digitalpathology'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./digitalpathology/X_test.npy
./digitalpathology/X.npy
./digitalpathology/Y.npy


In [2]:
from fastai.vision.all import *

## Intro

Last submission, I used the ```vit_small_r26_s32_224``` model with augmentation at training and TTA, and an item resize method of ```'squish'```. It improved accuracy from our initial starting point. However, to push the score up even more, I want to try a larger model, specifically the ```convnext_large_in22k``` model.

In [3]:
path = Path('./digitalpathology')
trn_x = np.load(path/'X.npy')
trn_y = np.load(path/'Y.npy')
trn_y_df = pd.DataFrame({'idx': np.arange(len(trn_y)), 'label': trn_y})

## Setting up Functions

In [4]:
def get_x(row):
    idx = row['idx']
    img_pix = trn_x[idx]
    img_pix = np.stack([img_pix]*3, axis=-1)
    return PILImage.create(img_pix.astype(np.uint8))

In [5]:
def train(arch,item,accum=1,epochs=5,finetune=False):
    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_x=get_x,
        get_y=ColReader('label'),
        item_tfms=item,
        batch_tfms=aug_transforms(size=224, min_scale=0.75))
    cbs = GradientAccumulation(64) if accum else []
    dls = dblock.dataloaders(trn_y_df, bs=64//accum)
    learn = vision_learner(dls, arch, metrics=accuracy, cbs=cbs).to_fp16()
    if finetune:
        learn.fine_tune(epochs)
        return learn
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs)

In [6]:
import gc
def clear_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

## Run the Model

In [7]:
learn = train(arch='convnext_large_in22k',
     item=Resize(224, method='squish'),
     accum=4, epochs=7, finetune=True)
clear_gpu()

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name convnext_large_in22k to current convnext_large.fb_in22k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/919M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,time
0,2.629126,0.512231,0.833333,01:36


epoch,train_loss,valid_loss,accuracy,time
0,0.657189,0.415829,0.848485,01:41
1,0.567310,0.188053,0.939394,01:41
2,0.383132,0.083265,0.984848,01:41
3,0.244810,0.045000,0.992424,01:41
4,0.178861,0.041053,0.984848,01:41
5,0.134551,0.039813,0.984848,01:41
6,0.094500,0.037551,0.992424,01:41


GPU:0
process       3132 uses     7332.000 MB GPU memory


In [8]:
learn.export('/kaggle/working/model.pkl')

## Make the Submission

In [9]:
def sub(n):
    tst_x = np.load(path/'X_test.npy')
    tst_dl = learn.dls.test_dl(tst_x)
    probs,_ = learn.tta(dl=tst_dl)
    idxs = np.argmax(probs, axis=1)
    preds = pd.Series(learn.dls.vocab[idxs], name='idxs')
    subm = pd.DataFrame({'Id': np.arange(300), 'Class': preds})
    subm.to_csv(f'sub{n}.csv', index=False)

In [10]:
sub(3)

/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.11/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


In [11]:
!head sub3.csv

Id,Class
0,4
1,16
2,2
3,18
4,11
5,5
6,6
7,4
8,8
